# Voedselprijzen in de derde wereld

Robert-Jan Korteschiel (10399143)  
Robert Houten  
Sander Kohnstamm (10715363)  
Joost de Wildt (12173002)  


## Vooronderzoek

Dit is een dataset die wekelijks wordt geupdated over voedselprijzen. De auteur is het World Food Programme, een humanitaire organisatie die steeft naar het ideaal van "zero hunger". Daar baseren ze al hun werk al op data en hebben ook al goede dataviz draaien. Interessant is dat ze hier combineren met veel GIS informatie zoals administratieve grenzen, wegennetwerken en klimaatdata. 

<div style="display: flex; width: 100%; justify-content: space-around;">
    <div style="width: 45%">
        <img src="food_viz.png" style="display: block; width: 100%; height: auto;" alt="GIS visualisation">
    </div>
    <div style="width: 45%">
        <img src="patterns.jpg" style="display: block; width: 100%; height: auto;" alt="Line visualisation">
    </div>
</div>


Er is dus al veel werk gedaan op de data, maar er mist ook veel informatie. Opvallend is dat economische en politieke gegevens missen in de analyse. Daarnaast bekijken ze de derde wereld echt per land en zien kijken ze bijna niet naar hoe deze problematiek grensoverstijgend is (of is dat wel zo?). Bovendien is het alles behalve een verhaal, dit is echt een visualisatie. Het is interessant of we delen kunnen namaken en tot een verhaal kunnen omvormen. 

[bron] https://data.humdata.org/dataset/wfp-food-prices  
[algemene omschriving] https://docs.wfp.org/api/documents/WFP-0000040024/download/   
[dierpere omschrijving] http://mvam.org/2018/11/20/getting-up-to-speed-wfp-food-data-on-hdx/  
[snelle verkenning] https://dataviz.vam.wfp.org   
[UN exchange rates] https://treasury.un.org/operationalrates/OperationalRates.php  


## Invalshoeken

1. Welke van de volgende gebeurtenissen heeft de grootste invloed op globale voedselprijzen?
    - Temperatuur
    - Brandstofprijs
    - Dagloon  


2. Vallen gebeurtenissen te herleiden uit voedselprijzen?



# Food

## Bootstrap

In [325]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn import preprocessing
from operator import itemgetter, attrgetter
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from functools import partial
import datetime
import math

init_notebook_mode(connected=True)

pd.options.display.max_rows = 100
pd.options.display.max_seq_items = 100

## Inladen

en laat de head() zien

In [263]:
food_df = pd.read_csv("./food_data/food.csv", low_memory=False)
display(food_df.head())

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


## Parse

### Groeperen

De vergelijkbaarheid per commodity per land echt heel laag, maar hele korte stukken data overlappen echt. Het WFP meet blijkbaar alleen als ze geinteresseerd zijn in iets, om de een of andere reden. Vervolgens redeneren we dat we al het voedsel samentrekken tot een enkele trend, dat moet de vergelijkbaarheid al aardig hoger maken. Voor onze onderzoeksvraag is het immers alleen interessant dat we veranderingen in voedselprijzen afzetten tegen brandstof, lonen en klimaat. Misschien dat we daarna gewoon een land kiezen met het meeste data, los van de vergelijkbaarheid, voor de andere onderzoeksvraag.

Groepeert alle commodities op basis het het eerste woord in hun definitie. 

**LET OP! Hier zit een grote assumptie. Een jaar kan een waarde krijgen op een enkele meting**

In [264]:
# groepeer de commodities met hun eerst woord
pat = '^([\w\-]+)'
selection = food_df["cm_name"].str.extract(pat, expand=False)

# run the foodtype string reduction
food_df["cm_name_selected"] = selection


### Parse time

Voeg een kolom toe met de date

In [265]:
dates = food_df[["mp_year", "mp_month"]].assign(day=1).rename(columns={"mp_year": "year", "mp_month": "month"})
food_df["dt"] = pd.to_datetime(dates)



## Transformatie

### Normalisatie

Normalisatie functie voor verschillende groepen in de data. Ik vermoed dat je het beste binnen een land op commodity kan normaliseren. Doe je het op een andere plek dan worden minder waardevolle commodities binnen groepen minder belangrijk. Het is aannemelijk dat graan wholesale flink goedkoper is dan graan retail. Als je die in absolute nummers optelt dan domineert het verschil in de retailprijs.

Je zou overigens nog een argument kunnen maken dat je eigenlijk op provincie of zelfs stadsniveau moet gaan normaliseren.


In [266]:
def normalize_group(key, group_df):
    # bootstrap a new normalizer   
    min_max_scaler = preprocessing.MinMaxScaler()
    
    # reshape the series to array so that the normalizer accepts it
    to_normalize = group_df[key].values.reshape(-1, 1)
    
    # do the actual normalisation     
    x_scaled = min_max_scaler.fit_transform(to_normalize)
    
    # undo some weird numpy nesting of arrays    
    x_scaled = np.concatenate(x_scaled).ravel()
    
    # concatenate it to the group_df    
    group_df[f"{key}_norm"] = x_scaled
    
    # trow the group out     
    return group_df
    
food_norm_df = food_df.groupby(by=["adm0_name", "cm_name"]).apply(partial(normalize_group, "mp_price"))


### Selecteren 

Verkenning en functies om te selecteren.

Spoiler: De groep "Rice" heeft de meeste datapunten en 2016 is het jaar met de minste NaN's.


#### Verkenning: Barchart

Maar welke goederen zijn daadwerkelijk interessant? Totaal aantal records of jaren per land met data.

**Dee laatste kwaliteit metriek gebruik ik in de rest van de selecties, met toevoeging van brandstof en wage**

In [267]:

# run a groupby for use later on
grouped_commodity_df = food_norm_df.groupby(by=["adm0_name", selection, "mp_year"])

def barchart_records():
    # calculate
    quality_count = grouped_commodity_df["mp_price"].count().reset_index().groupby(by="cm_name")["mp_price"].sum().sort_values()

    # truncate the long barchart
    quality_count_trunc = quality_count[-20:]

    # visualize
    barchart_trace = go.Bar(y=quality_count_trunc.index, x=quality_count_trunc.values, orientation = 'h')

    layout = go.Layout(
        autosize=False,
        width=800,
        height= 200 + 20 * len(quality_count_trunc.index),
        yaxis=dict(
            showticklabels=True,
            automargin=True
        )
    )

    fig = go.Figure(data=[barchart_trace], layout=layout)
    print(f"Total sum of records: {quality_count.sum()} " )
    iplot(fig)

def barchart_years():
    quality_years = grouped_commodity_df["mp_price_norm"].mean().reset_index().groupby(by=["cm_name"])["mp_price_norm"].count().sort_values()
    quality_years_trunc = quality_years[-20:]

    barchart_trace = go.Bar(y=quality_years_trunc.index, x=quality_years_trunc.values, orientation = 'h')

    layout = go.Layout(
        autosize=False,
        width=800,
        height= 200 + 20 * len(quality_years_trunc.index),
        yaxis=dict(
            showticklabels=True,
            automargin=True
        )
    )

    fig = go.Figure(data=[barchart_trace], layout=layout)
    print(f"Total sum of year with data: {quality_years.sum()} " )
    iplot(fig)

def barswitcher(barchart_type):
    if barchart_type == "total records":
        barchart_records()
    if barchart_type == "per year, per country":
        barchart_years()

interact(barswitcher, barchart_type=["total records", "per year, per country"])

# Doe de eerste tien en concatenate wage (fuel zit al in deze lijst)
selectie = [*grouped_commodity_df["mp_price_norm"].mean().reset_index().groupby(by=["cm_name"])["mp_price_norm"].count().sort_values(ascending=False)[:10].index, "Wage"]


interactive(children=(Dropdown(description='barchart_type', options=('total records', 'per year, per country')…

#### Verkenning: Heatmap

Even een beetje verkenning om wat beter zicht op de data te krijgen. In deze visualisatie kun je goed zien waar het gewicht in de data zit. Hier zie je goed hoe een goed per land zit. De barcharts geven slecht totalen aan, hier zie je de daarwerkelijke verdeling.

In [268]:
def heatmap_quality(yaxis, zaxis, commodity, country):
    print("TOTAL MEASUREMENTS: ", grouped_commodity_df["mp_price_norm"].count().sum())
    if zaxis == "count":
        quality = grouped_commodity_df["mp_price_norm"].count()
        zmax = 100
        zmin = 0
        colorscale = "Viridis"
    elif zaxis == "price_norm":
        quality = grouped_commodity_df["mp_price_norm"].mean()
        zmax = 1
        zmin = -1
        colorscale = "Portland"
    else:
        quality = grouped_commodity_df["mp_price_norm"].mean().groupby(level=[0,1]).diff()
        zmax = 0.4
        zmin = -0.4
        colorscale = "Portland"
    
    if yaxis == "commodity":
        quality = quality.xs(commodity, level=1).unstack(level=1)
        quality = quality.reindex(sorted(quality.columns), axis=1)
    
    elif yaxis == "country":
        quality = quality.xs(country, level=0).unstack(level=1)
        quality = quality.reindex(sorted(quality.columns), axis=1)

    heatmap_trace = go.Heatmap(z = quality.values, x=quality.columns, y=quality.index, colorscale= colorscale, zmin=zmin, zmax=zmax)
    
    layout = go.Layout(
        autosize=False,
        width=800,
        height= 200 + 20 * len(quality.index),
        xaxis={"range": [1990, 2019]},
        yaxis=dict(
            showticklabels=True,
            automargin=True
        ),
        title=commodity if yaxis == "commodity" else country
    )

    
    fig = go.Figure(data=[heatmap_trace], layout=layout)
    print("EITHER THE COMMODITY OR COUNTRY VARIABLE IS RENDERED, NOT COMBINATIONS")
    iplot(fig)

    
nestedInteract = interact(heatmap_quality, yaxis=["commodity", "country"], zaxis=["count", "price_norm", "price_norm_diff"], commodity=[*selectie], country=[*food_norm_df["adm0_name"].unique()])



interactive(children=(Dropdown(description='yaxis', options=('commodity', 'country'), value='commodity'), Drop…

#### Verkenning: Outliers scatterplots

Ik gooi heel doodleuk .mean().diff() op de dataset. Klaar, paar transformaties ertussen en huppathee. Maar de mean is geen robuuste metriek van centraliteit. Even iets meer zicht krijgen op de lijnen die ik aan het reduceren ben tot een gemiddelde.

Eerst even wat prep:

In [345]:
def scatter_commodity_country(norm, country, commodity):
    price_type = "mp_price_norm" if norm else "mp_price"

    # select a country
    if country == "all":
        country_food_points = food_norm_df
    else:
        country_food_points = food_norm_df[food_norm_df["adm0_name"] ==
                                           country]

    if commodity == "all":
        pass
    else:
        country_food_points = country_food_points[
            country_food_points["cm_name_selected"] == commodity]

    # make a scatter
    scatter_trace = go.Scattergl(x=country_food_points["dt"],
                                 y=country_food_points[price_type],
                                 mode='markers',
                                 name="Meetpunten",
                                 marker=dict(color="#2077b4",
                                             size=2,
                                             line=dict(width=0)))

    # make a mean
    county_food_mean = country_food_points.groupby(
        by=[country_food_points.dt.dt.year])[price_type].mean()
    line_trace = go.Scatter(x=county_food_mean.index,
                            y=county_food_mean.values,
                            name="Mean")

    # make a median
    county_food_median = country_food_points.groupby(
        by=[country_food_points.dt.dt.year])[price_type].median()
    median_trace = go.Scatter(x=county_food_median.index,
                              y=county_food_median.values,
                              name="Mediaan")

    print("Unique commodities: ",
          list(country_food_points["cm_name"].unique()))

    fig_scatter = go.Figure(data=[scatter_trace, median_trace, line_trace])
    fig_scatter['layout'].update(title=f"{country}: {commodity}")
    iplot(fig_scatter)

    # Histogram construction         
    histo_years = country_food_points["mp_year"].unique()
    fig_histo = tools.make_subplots(rows=math.ceil(len(histo_years) / 3), cols=3, print_grid=False)
    
    for i in range(len(histo_years)):
        histo_data_year = country_food_points[(country_food_points["mp_year"] == histo_years[i])]
        histo_trace = go.Histogram(x=histo_data_year["mp_price_norm"],
                                   xbins=dict(
                                       start=0,
                                       end=1,
                                       size=0.05,
                                   ),
                                   name=f"{histo_years[i]}"
                                    )
        
        fig_histo.append_trace(histo_trace,  math.ceil((i + 1) / 3 ), i % 3 + 1)
    
    iplot(fig_histo)

    # add some descriptives
#     descriptives = country_food_points.groupby(by=["mp_year"])[price_type].describe()
#     descriptives = descriptives.append(country_food_points[price_type].describe().rename("Total"))
#     descriptives["coefficient of variation"] = descriptives["std"] / descriptives["mean"]
#     display(descriptives)

interact(scatter_commodity_country,
         norm=[True, False],
         country=[*food_norm_df["adm0_name"].unique(), "all"],
         commodity=[*selectie, "all"])

interactive(children=(Dropdown(description='norm', options=(True, False), value=True), Dropdown(description='c…

<function __main__.scatter_commodity_country(norm, country, commodity)>

#### Verkenning: Graag alle patronen tegelijkertijd

**WIP!! NIET AF!**

Wat doen een mean over alle landen? Die we uiteindelijk nodig hebben om onze hypotheses te toetsen. Welke landen worden grof fout gerepresenteerd in de lijn? Grid met lijngrafieken met land en gemiddelde van alle landen.



In [270]:
def scatter_commodity_country(country, commodity):
    # construct global mean WRONG!!!!!!!
#     global_mean = food_selected_df[(food_selected_df["adm0_name"] == country)]["mp_price"].groupby(by=[food_selected_df.dt.dt.year]).mean()
#     display(global_mean)
    
    # select a country
    country_food_points = food_selected_df[(food_selected_df["adm0_name"] == country) & (food_selected_df["cm_name"] == commodity)]
    
    # make a mean
    county_food_mean = country_food_points.groupby(by=[country_food_points.dt.dt.year])["mp_price"].mean()
    county_mean_trace = go.Scatter(
        x = county_food_mean.index,
        y = county_food_mean.values,
        name = "Mean"
    )
    
    # global mean    
    
    iplot([county_mean_trace])

interact(scatter_commodity_country, country=food_selected_df["adm0_name"].unique(), commodity=selectie)


NameError: name 'food_selected_df' is not defined

### Reduceren

Het gemiddelde berekenen en het verschil per jaar. 

In [ ]:
# bereken het gemiddelde per jaar en vergelijk dat met het vorige jaar
grouped_commodity_mean_df = grouped_commodity_df["mp_price_norm"].mean()

# unstack zodat het verschil per jaar berekend kan worden
grouped_commodity_diff_df = grouped_commodity_mean_df.unstack(level=2).diff(axis=1)
grouped_commodity_diff_df.head()

## Utility

Even voor het gemak

In [ ]:
# Utility om selecteren makkelijker te maken
def select_commodity(commodity, year):
    return grouped_commodity_diff_df.xs(commodity, level=1, drop_level=True)[year]

## Visualisatie: Map

Okee, een kaart met interactiviteit. Vond de Iphyton docs eigenlijk fijner.

[interactieve plots] https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6  
[iphython widget docs] https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

### Food change maps

In [ ]:
def map_graph(commodity=selectie, year=(2007, 2019, 1)):
    # Plot the mean price, of all commodities on map, of all years.
    series = select_commodity(commodity, year)
    map_data = [go.Choropleth(
        locations = series.index,
        locationmode = "country names",
        z = series.values,
    )]

    fig = go.Figure(data = map_data)
    iplot(fig)
    
interact(map_graph, commodity=selectie, year=(1996, 2019, 1))
interact(map_graph, commodity=selectie, year=(1996, 2019, 1))


## Controle

Om te controleren of de dataset correct is moeten we natuurlijk even een test draaien. Ik heb globale voedselprijzen even als benchmark genomen. Daar ligt waarschijnlijk een sterke correlatie, zeker voor gebieden die niet direct in conflict zijn.

[imf commodity prices data] https://www.imf.org/en/Research/commodity-prices  
[interessante analyse voedsel] https://ourworldindata.org/food-prices  
[interessante statistische analyse van commodities] https://www.imf.org/~/media/Files/Research/CommodityPrices/WEOSpecialFeature/SFApril2019.ashx  


In [ ]:
# selecteer rijst en bereken het gemiddelde
commodity_imf_df = pd.read_csv("./commodity_imf_proper.csv", dtype=float, decimal=",")
commodity_imf_df = commodity_imf_df.set_index('Year')

def normalize_series(series):
    # bootstrap a new normalizer   
    min_max_scaler = preprocessing.MinMaxScaler()
    
    # reshape the series to array so that the normalizer accepts it
    to_normalize = series.values.reshape(-1, 1)
    
    # do the actual normalisation     
    x_scaled = min_max_scaler.fit_transform(to_normalize)

    # undo some weird numpy nesting of arrays    
    x_scaled = np.concatenate(x_scaled).ravel()
    
    # trow the series     
    return pd.Series(x_scaled, index = series.index)

def line_graph(food):
    # create an average of all countries    
    test = grouped_commodity_diff_df.xs(food, level=1, drop_level=False).mean()
    
    # use the IMF data to compute a nomalized and diffed line too    
    control = normalize_series(commodity_imf_df[food]).diff()
    
    test_trace = go.Scatter(
        name = "WFP",
        x = test.index,
        y = test.values
    )
    
    control_trace = go.Scatter(
        name = "IMF",
        x = control.index[-25:],
        y = control.values[-25:]
    )
    
    iplot([test_trace, control_trace], filename='basic-line')

interact(line_graph, food=["Rice", "Wheat", "Sugar"])



## Resultaat

Er lijkt wel iets van een correlatie te zijn, maar er zitten ook rare verschillen tussen. Metname de pieken zijn veel hoger. Maar duidelijk is dat 2008 een grote impact heeft gehad. Ik denk dat we nog even goed moeten kijken naar hoe we normaliseren. Het is best mogelijk om hiermee te spelen lijkt het, maar wat verantwoord is twijfel ik aan.

# Klimaat

Nu eens kijken of het samenhangt met klimaat. Ik heb het gevoel dat hier enige correlatie vinden hier moeilijk gaat worden.

## Load data

In [ ]:

climate_df = pd.read_csv("./climate_data/GlobalLandTemperaturesByCountry.csv", low_memory=False)
display(climate_df.head())


## Type and normalize

Here we go again.

1. Type the datetime
2. Normalize
3. Create year averages
4. Diff those averages to have something comparable

This is just a tad different data though. It's cumulative data, its not the same as money. And the effects are much longer term. Doesn't make sense.

In [ ]:
# type properly
climate_df_typed = climate_df.copy()
climate_df_typed['dt'] = climate_df_typed['dt'].astype('datetime64[ns]')

# apply the normalize function (with a partial for the key)
climate_norm_df = climate_df_typed.groupby(by=["Country"]).apply(partial(normalize_group, "AverageTemperature"))


## Transform and select

In [ ]:
# calulate the mean for the year
climate_mean_df = climate_norm_df.groupby(by=["Country", climate_norm_df["dt"].dt.year]).mean()
climate_mean_df.reset_index(inplace=True)

# quick function with assignment
def diff_temp(group_df):
    group_df["AverageTemperature_diff"] = group_df["AverageTemperature_norm"].diff()
    return group_df

# apply the difference function
climate_tran_df = climate_mean_df.groupby(by=["Country"]).apply(diff_temp).set_index(["Country", "dt"])

# quick selection function
def select_year(year):
    return climate_tran_df.xs(year, level=1, drop_level=False)

## Map: Tempratuur verschil met vorig jaar

Spreekt voor zich, maar of het interessant is is de tweede vraag.

In [ ]:
def map_climate(year):
    year_data = select_year(year)
    map_data = go.Choropleth(locations=year_data.index.get_level_values(0),
                             locationmode="country names",
                             z=year_data["AverageTemperature_diff"].values)
    
    fig = go.Figure(data=[map_data])

    iplot(fig)

interact(map_climate, year=(1996, 2013, 1))



# Sparklines Climate

Misschien is een totaalbeeld beter. 

In [ ]:
import plotly.plotly as py
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/1962_2006_walmart_store_openings.csv')
display(df.head())

data = []
layout = dict(
    title = 'New Walmart Stores per year 1962-2006<br>\
Source: <a href="http://www.econ.umn.edu/~holmes/data/WalMart/index.html">\
University of Minnesota</a>',
    # showlegend = False,
    autosize = False,
    width = 1000,
    height = 900,
    hovermode = False,
    legend = dict(
        x=0.7,
        y=-0.1,
        bgcolor="rgba(255, 255, 255, 0)",
        font = dict( size=11 ),
    )
)
years = df['YEAR'].unique()

for i in range(len(years)):
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    lons = list(df[ df['YEAR'] == years[i] ]['LON'])
    lats = list(df[ df['YEAR'] == years[i] ]['LAT'])
    # Walmart store data
    data.append(
        dict(
            type = 'scattergeo',
            showlegend=False,
            lon = lons,
            lat = lats,
            geo = geo_key,
            name = str(years[i]),
            marker = dict(
                color = "rgb(0, 0, 255)",
                opacity = 0.5
            )
        )
    )
    # Year markers
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-78],
            lat = [47],
            geo = geo_key,
            text = [years[i]],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
        scope = 'usa',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = False,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )


def draw_sparkline( domain, lataxis, lonaxis ):
    ''' Returns a sparkline layout object for geo coordinates  '''
    return dict(
        showland = False,
        showframe = False,
        showcountries = False,
        showcoastlines = False,
        domain = domain,
        lataxis = lataxis,
        lonaxis = lonaxis,
        bgcolor = 'rgba(255,200,200,0.0)'
    )

# Stores per year sparkline
layout['geo44'] = draw_sparkline({'x':[0.6,0.8], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 30.0]}, {'range':[0.0, 40.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df.groupby(by=['YEAR']).count()['storenum']/1e1),
        lon = list(range(len(df.groupby(by=['YEAR']).count()['storenum']/1e1))),
        line = dict( color = "rgb(0, 0, 255)" ),
        name = "New stores per year<br>Peak of 178 stores per year in 1990",
        geo = 'geo44',
    )
)

# Cumulative sum sparkline
layout['geo45'] = draw_sparkline({'x':[0.8,1], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 50.0]}, {'range':[0.0, 50.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df.groupby(by=['YEAR']).count().cumsum()['storenum']/1e2),
        lon = list(range(len(df.groupby(by=['YEAR']).count()['storenum']/1e1))),
        line = dict( color = "rgb(214, 39, 40)" ),
        name ="Cumulative sum<br>3176 stores total in 2006",
        geo = 'geo45',
    )
)

z = 0
COLS = 5
ROWS = 9
for y in reversed(range(ROWS)):
    for x in range(COLS):
        geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
        layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
        layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
        z=z+1
        if z > 42:
            break


layout = {**layout, "height":900, "width":1000 }
fig = { 'data':data, 'layout': layout}
iplot( fig, filename='US Walmart growth' )

# Events

Okee, wat moet ik hiervoor doen. Querys schrijven naar Google Big Query. Cool, eens kijken hoe dat werkt.

[GDELT database] https://www.gdeltproject.org/  
[forecasting op voedseldata] https://dataviz.vam.wfp.org/economic_explorer/price-forecasts-alerts  
[redelijke uitleg GDELT] http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf  
[Uitleg gebruikte codes] https://www.gdeltproject.org/data/documentation/CAMEO.Manual.1.1b3.pdf  

## Download data

Er is een Python client voor bigquery. Het is best simpel eigenlijk.

1. De clientlib installeren
2. Credentials file exporten in je shell (ik heb het aan mijn ~/.bash_profile toegevoegd en die gesourced, werkt perfect - MAC/Linux-only helaas)
2. Draai de sample query
3. Doe dingen met het dataframe

[setup credentials] https://cloud.google.com/bigquery/docs/reference/libraries#client-libraries-install-python  
[database description] https://bigquery.cloud.google.com/table/gdelt-bq:gdeltv2.events?pli=1  
[API reference] https://googleapis.github.io/google-cloud-python/latest/bigquery/index.html

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

# write a good query to the right table (lookup the database stuffz)
query = (
    "SELECT V2Themes FROM `gdelt-bq.gdeltv2.gkg` WHERE DATE>20150302000000 and DATE < 20150304000000 and V2Persons like '%Netanyahu%' LIMIT 10;"
)

# create the query
query_job = client.query(
    query,
    location="US",
)

# run the query and turn the result to a dataframe
result = query_job.result().to_dataframe()

## Query

Wat moet het kunnen?

Het moet sentiment en count per land per jaar of maand geven.


In [ ]:
display(result)

# Todo

## Food
Dit is onze hoofddataset, dus veel detailvisualisaties.

0. [ x ] Drop messy date columns
  
1. [ x ] Scatter transform van price naar price_norm  
  
2. [ x ] Matrix   
    2.1. [ x ] Matrix transform interactief/y-as van goods naar country    
    2.2. [ x ] Matrix transform z-as van count naar price en price_norm  
  
3. [  ] Gridplot van een commodity per land om een diachroon perspectief te krijgen  
3.1. [  ] mp_price/jaar  
3.2. [  ] mp_price_norm/jaar  
  
4. [  ] Totaalplot  
    4.1. [  ] IMF Wereldcommodities aanvullen met scatterplot  
    4.2. [  ] Correlatiematrix binnen een commodity in een land maken  
    4.3. [  ] Slechte correlaties droppen  
    4.4. [  ] Correlatiematrix tussen dezelfde commodity tussen landen  
    4.5. [  ] Slechte correlaties droppen   
    4.6. [  ] Correlatie met wereldcommodities maken  

5. [  ] Correlatie plots
    5.1. [  ] Correlation scatter grid
    5.2. [  ] Correlation diagram

5. [  ] Detail in alle grafieken naar maanden verhogen  
5.1. [  ] Matrix  
5.2. [  ] Lijn  
   
6. [  ] Maps naar provincie of stadsniveau  



## Klimaat

### EDA
1. Dit is het zwakke van klimaat, ik heb eigenlijk nog geen goed overzicht van distributes en ander inzicht in de kwaliteit van die set. 
2. Iets van diachronaal perspectief op deze set, om uberhaubt te checken dat het geen nagenoeg rechte lijn van 1990-2013 is. 



## Events

### Opzetten

#### Gemiddelden checken 
1. Goede query opstellen. Dit wordt eigenlijk de grote crux. In principe moeten we BigQuery al het werk laten doen hier. Het resultaat dat we willen hebben is niet zo heel erg gedetailleerd. 
2. Transformeren naar een vergelijkbaar dataframe. 
3. Plotten in lijn en kaart

#### Individuele events checken
4. Eventdetectie in de food dataset
5. Event en food events tegen elkaar afzetten

### EDA
6. Alles

### Kwaliteit
7. Wat de data precies beschrijven en wat de bronnen zijn twijfel ik ook nog aan. Het is regelrecht gigantisch en het is een academisch project. Maar dit is wel een extreem complexe set, met veel artikelen over hoe het gecodeerd is. (denk Goldstein sentiment en counts.. van wat precies?)



## Correlaties
1. Proberen goederen te clusteren tot een enkele lijn per land / wereld
2. Visuele EDA op de correlaties
3. Correlaties van die lijn inferentieel onderzoeken en tot een coefficient omzetten voor uiteindelijke conclusie.
    - Nadenken over welke assumpties daar ook al weer voor golden en welke tests dus aan voldaan moet zijn.


